In [58]:
import os
import sys
import altair as alt
import pandas as pd
import itertools as it

# input_folder = "/home/sgro/data/output_robertson_2/"
# input_folder = "/data/gen-data/output_robertson_2/"
# input_folder = "../OUTPUT/"
input_folder = "/Users/msgro/Projects/Globus/OUTPUT-new/"
sample_list_txt = "../DATA/samples.txt"
plots = "./plots"
try:
    os.mkdir(plots)
except:
    print("warning, plots directory exists")
## LETTURA METRICHE DI LABELING, .stats.txt file
### PER CUT
range_ = ['#ffcdb2', '#B3DFFA',
          '#ffb4a2', '#9ABFD6',
          '#e5989b', '#7D9BAD',
          '#b5838d', '#5F7685']
range_3 = ['#ffcdb2', '#B3DFFA', "#D8FA8E",
          '#ffb4a2', '#9ABFD6', "#B9D67A",
          '#e5989b', '#7D9BAD', "#96AD63",
          '#b5838d', '#5F7685', "#73854C"]

warning, plots directory exists


In [31]:
cuts = [0, 100, 250, 500]
pbf_graphs = ['uni', 'ske', 'panstar.uni', 'panstar.ske', 'pan.uni', 'pan.ske']
gplas_graphs = ['uni', 'ske', 'pan.uni', 'pan.ske']

eval_string = {}

for x in cuts:
    for y in pbf_graphs:
        eval_string.update({f"{str(x) + ".pbf." + y}": []})
for x in cuts:
    for y in gplas_graphs:
        eval_string.update({f"{str(x) + ".gplas." + y}": []})
print(eval_string)
# for key in eval_string:

stat_map  = {"prec":[], "rec":[], "f1":[]}
stat_tests = {}
for test in eval_string:
    stat_tests.update({test: stat_map})
    
print(stat_tests)



dataset = []
with open(sample_list_txt, mode='r') as f:
    for lines in f:
        sample = lines.strip()
        dataset.append( f"{sample}")
        
print(dataset)
print(len(dataset), "input samples")

actual_dataset = []
actual_dataset_files = []

performed_tests_files = {}
performed_tests = {}

# for subfolder in os.listdir(input_folder):
#     if not str(subfolder).startswith("."):
#         actual_dataset.append(f"{subfolder}")
for root, dirs, files in os.walk(input_folder):# + subfolder):
    for file in files:
        if file.endswith("stats.txt"): 
            evaluation = str(file).split(".")
            sample_split = evaluation[0].split("-")
            sample_name = sample_split[-2]
            sample_cut = sample_split[-1]
            eval_type = str(sample_cut) + '.' + '.'.join(evaluation[1:len(evaluation)-2])
            # print("sample:", sample_name, "\n", eval_type, "\n ------- ")
            # print(eval_type)
            eval_sample_list = eval_string.get(eval_type)
            
            eval_string.update({eval_type: eval_sample_list + [sample_name] })
            old_list = performed_tests.get(sample_name)
            old_list_files = performed_tests_files.get(sample_name)
            file_path = os.path.abspath(os.path.join(root, file))
            if old_list == None:
                performed_tests.update({sample_name: [eval_type]})
                performed_tests_files.update({sample_name: [file_path]})
            else:
                performed_tests.update({sample_name: old_list + [eval_type]})
                performed_tests_files.update({sample_name: old_list_files + [file_path]})
                
            with open(file_path, "r") as f:
                lines = f.readlines()
            print(lines)
            prec, rec, f1 = float(lines[-4].split("\t")[-1].strip()), float(lines[-3].split("\t")[-1].strip()), float(lines[-2].split("\t")[-1].strip())
            
            prec_list, rec_list, f1_list = [],[],[]
            prec_list = stat_tests.get(eval_type).get('prec')
            rec_list = stat_tests.get(eval_type).get('rec')
            f1_list = stat_tests.get(eval_type).get('f1')

            stat_tests.update({eval_type: {'prec': prec_list + [prec],
                                           'rec': rec_list + [rec],
                                           'f1': f1_list + [f1]}})

            
                ## PANGENOME vs assembl
# print(subfolder)

# print(actual_dataset)
# print(len(actual_dataset), "evaluated samples")
print(performed_tests)
print(performed_tests_files)


{'0.pbf.uni': [], '0.pbf.ske': [], '0.pbf.panstar.uni': [], '0.pbf.panstar.ske': [], '0.pbf.pan.uni': [], '0.pbf.pan.ske': [], '100.pbf.uni': [], '100.pbf.ske': [], '100.pbf.panstar.uni': [], '100.pbf.panstar.ske': [], '100.pbf.pan.uni': [], '100.pbf.pan.ske': [], '250.pbf.uni': [], '250.pbf.ske': [], '250.pbf.panstar.uni': [], '250.pbf.panstar.ske': [], '250.pbf.pan.uni': [], '250.pbf.pan.ske': [], '500.pbf.uni': [], '500.pbf.ske': [], '500.pbf.panstar.uni': [], '500.pbf.panstar.ske': [], '500.pbf.pan.uni': [], '500.pbf.pan.ske': [], '0.gplas.uni': [], '0.gplas.ske': [], '0.gplas.pan.uni': [], '0.gplas.pan.ske': [], '100.gplas.uni': [], '100.gplas.ske': [], '100.gplas.pan.uni': [], '100.gplas.pan.ske': [], '250.gplas.uni': [], '250.gplas.ske': [], '250.gplas.pan.uni': [], '250.gplas.pan.ske': [], '500.gplas.uni': [], '500.gplas.ske': [], '500.gplas.pan.uni': [], '500.gplas.pan.ske': []}
{'0.pbf.uni': {'prec': [], 'rec': [], 'f1': []}, '0.pbf.ske': {'prec': [], 'rec': [], 'f1': []}, '0

In [32]:

header = ["sample", "tool", "precision", "recall", "f1"]
df = pd.DataFrame(columns=header)
for test in eval_string:
    for i in range(len(eval_string[test])):
        assert (len(eval_string[test]) == len(stat_tests[test]['rec'])),f"{len(eval_string[test]), "\n", len(stat_tests[test]['prec'])}" + "\n" + f"{eval_string[test]}" + "\n" + f"{stat_tests[test]['prec']}"
        df.loc[len(df)] = {"sample":eval_string[test][i], "tool":test, "precision":stat_tests[test].get('prec')[i], "recall":stat_tests[test].get('rec')[i], "f1":stat_tests[test].get('f1')[i]}
        
print(df)
df.to_csv('results.csv',sep='\t')
df_bcp = df.copy()
df = df.query("precision > 0.0 & recall > 0.0 & f1 > 0.0")
df.sort_values('sample').to_csv('results.clean.csv',sep='\t', index=False)



            sample               tool  precision    recall        f1
0     SAMEA3357080          0.pbf.uni   0.664530  0.981018  0.792339
1     SAMN07312490          0.pbf.uni   0.947784  0.989622  0.968251
2     SAMEA3357223          0.pbf.uni   0.277703  0.986810  0.433432
3     SAMN04014852          0.pbf.uni   0.952899  0.790494  0.864132
4     SAMN03024573          0.pbf.uni   0.983441  0.989688  0.986555
...            ...                ...        ...       ...       ...
1718  SAMN16357465  500.gplas.pan.ske   1.000000  0.787086  0.880860
1719  SAMN16357464  500.gplas.pan.ske   0.998305  0.016905  0.033247
1720  SAMN16357455  500.gplas.pan.ske   1.000000  0.050686  0.096482
1721  SAMN11056344  500.gplas.pan.ske   0.994637  0.645038  0.782568
1722  SAMEA3357268  500.gplas.pan.ske   0.879360  0.056071  0.105420

[1723 rows x 5 columns]


In [33]:
for test in eval_string:
    final_performed = len(eval_string[test])
    if final_performed > 0:
        print("for test", test, "completed:", final_performed)

    

for test 0.pbf.uni completed: 49
for test 0.pbf.ske completed: 47
for test 0.pbf.panstar.uni completed: 48
for test 0.pbf.panstar.ske completed: 48
for test 0.pbf.pan.uni completed: 50
for test 0.pbf.pan.ske completed: 50
for test 100.pbf.uni completed: 49
for test 100.pbf.ske completed: 48
for test 100.pbf.panstar.uni completed: 46
for test 100.pbf.panstar.ske completed: 46
for test 100.pbf.pan.uni completed: 46
for test 100.pbf.pan.ske completed: 46
for test 250.pbf.uni completed: 49
for test 250.pbf.ske completed: 49
for test 250.pbf.panstar.uni completed: 47
for test 250.pbf.panstar.ske completed: 47
for test 250.pbf.pan.uni completed: 48
for test 250.pbf.pan.ske completed: 48
for test 500.pbf.uni completed: 47
for test 500.pbf.ske completed: 49
for test 500.pbf.panstar.uni completed: 48
for test 500.pbf.panstar.ske completed: 48
for test 500.pbf.pan.uni completed: 46
for test 500.pbf.pan.ske completed: 46
for test 0.gplas.uni completed: 45
for test 0.gplas.ske completed: 44
for te

In [55]:
for test in eval_string:
    df_tool = df.loc[df['tool'] == test]
    
    charts = {"prec":None, "rec":None, "f1":None}
    leg = alt.Legend(title="Input Type", orient="right")
    chub = alt.BarConfig(strokeWidth=0.5, stroke="black")

    charts["prec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("precision:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg).scale(scheme="dark2"),
    )

    charts["rec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("recall:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    charts["f1"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("f1:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"Robertson Benchmark Binning Scores ({len(eval_string[test])} samples)").configure_axis(
        tickRound=False
    )
    mixed.save(f"{plots}/labeling.{test}.pdf", ppi=300)


In [60]:
pairs =[
    [
        '0.gplas.uni',   
        '100.gplas.uni', 
        '250.gplas.uni',
        '500.gplas.uni',
        
        '0.gplas.pan.uni',
        '100.gplas.pan.uni',
        '250.gplas.pan.uni',
        '500.gplas.pan.uni' 
    ],
    [
        '0.gplas.ske', '0.gplas.pan.ske',
        '100.gplas.ske', '100.gplas.pan.ske',
        '250.gplas.ske', '250.gplas.pan.ske',
        '500.gplas.ske', '500.gplas.pan.ske'
    ],
    [
        '0.pbf.uni', '0.pbf.pan.uni', '0.pbf.panstar.uni',
        '100.pbf.uni', '100.pbf.pan.uni', '100.pbf.panstar.uni',
        '250.pbf.uni', '250.pbf.pan.uni', '250.pbf.panstar.uni',
        '500.pbf.uni', '500.pbf.pan.uni', '500.pbf.panstar.uni'
    ],
    [
        '0.pbf.ske', '0.pbf.pan.ske', '0.pbf.panstar.ske',
        '100.pbf.ske', '100.pbf.pan.ske', '100.pbf.panstar.ske',
        '250.pbf.ske', '250.pbf.pan.ske', '250.pbf.panstar.ske',
        '500.pbf.ske', '500.pbf.pan.ske', '500.pbf.panstar.ske'
    ],

]
for i in range(0,4):
    if i in [0,1]: color = range_
    if i in [2,3]: color = range_3

    df_tool = df.loc[df['tool'].isin(pairs[i])]

    charts = {"prec":None, "rec":None, "f1":None}
    leg = alt.Legend(title="Input Type", orient="right")
    chub = alt.BarConfig(strokeWidth=0.5, stroke="black")

    charts["prec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("precision:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg).scale(range=color),
    )

    charts["rec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("recall:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    charts["f1"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("f1:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"Robertson Benchmark Binning Scores ({len(performed_tests)} samples)").configure_axis(
        tickRound=False
    )

        
    mixed.save(f"labeling.comparing.assembly.{i}.pdf", ppi=300)





In [62]:
pairs =[
    [
        '0.pbf.uni', 
        '0.gplas.uni', 
        '100.pbf.uni', 
        '100.gplas.uni', 
        '250.gplas.uni',
        '250.pbf.uni', 
        '500.pbf.uni',
        '500.gplas.uni',
        

    ],
    [
        '0.gplas.ske', 
        '100.gplas.ske',
        '250.gplas.ske',
        '500.gplas.ske',
                '500.pbf.ske',
                        '250.pbf.ske',

        '100.pbf.ske',
                '0.pbf.ske',



    ],
    [
        '0.gplas.pan.uni',
        '100.gplas.pan.uni',
        '250.gplas.pan.uni',
        '500.gplas.pan.uni',
        '0.pbf.pan.uni', '0.pbf.panstar.uni',
        '100.pbf.pan.uni', '100.pbf.panstar.uni',
        '250.pbf.pan.uni', '250.pbf.panstar.uni',
        '500.pbf.pan.uni', '500.pbf.panstar.uni'
    ],
    [
        '0.pbf.pan.ske', '0.pbf.panstar.ske',
        '100.pbf.pan.ske', '100.pbf.panstar.ske',
        '250.pbf.pan.ske', '250.pbf.panstar.ske',
        '500.pbf.pan.ske', '500.pbf.panstar.ske',
        '0.gplas.pan.ske',
         '100.gplas.pan.ske',
          '250.gplas.pan.ske',
     '500.gplas.pan.ske'
     ]

]
for i in range(0,4):
    if i in [0,1]: color = range_
    if i in [2,3]: color = range_3
    df_tool = df.loc[df['tool'].isin(pairs[i])]

    charts = {"prec":None, "rec":None, "f1":None}
    leg = alt.Legend(title="Input Type", orient="right")
    chub = alt.BarConfig(strokeWidth=0.5, stroke="black")

    charts["prec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("precision:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg).scale(range=color)
    )

    charts["rec"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("recall:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    charts["f1"] = alt.Chart(df_tool).mark_boxplot(box=chub, rule=chub).encode(
        alt.X("tool:N", title=None, axis=alt.Axis(labels=False)),
        alt.Y("f1:Q").scale(zero=False),
        alt.Color("tool:N").legend(leg)
    )

    mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"Robertson Benchmark Binning Scores ({len(performed_tests)} samples)").configure_axis(
        tickRound=False
    )

        
    mixed.save(f"labeling.comparing.tools.{i}.pdf", ppi=300)





In [8]:
# FIRST SCATTER SERIES

header = ["sample", "pbf_uni", "pbf_skesa", "pangebin", "gplas", "metric"]
df_scatter = pd.DataFrame(columns=header)
for i in range(len(skesa_stats["prec"])):
    df_scatter.loc[len(df_scatter)] = {"sample":i,"pbf_uni":uni_stats["prec"][i], "pbf_skesa":skesa_stats["prec"][i], "pangebin":pange_mod1["prec"][i], "gplas":gplas_stats["prec"][i], "metric":"precision"}
    df_scatter.loc[len(df_scatter)] = {"sample":i,"pbf_uni":uni_stats["rec"][i], "pbf_skesa":skesa_stats["rec"][i], "pangebin":pange_mod1["rec"][i], "gplas":gplas_stats["rec"][i], "metric":"recall"}
    df_scatter.loc[len(df_scatter)] = {"sample":i,"pbf_uni":uni_stats["f1"][i], "pbf_skesa":skesa_stats["f1"][i], "pangebin":pange_mod1["f1"][i], "gplas":gplas_stats["f1"][i], "metric":"f1"}
    
charts = {"prec":None, "rec":None, "f1":None}
leg = alt.Legend(title="Input Type", orient="right")
chub = alt.BarConfig(strokeWidth=0.5, stroke="black")
# header = ["sample", "value", "metric"]

charts["prec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_skesa:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='precision')
)


charts["rec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_skesa:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='recall')
)

charts["f1"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_skesa:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='f1')
)

mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"[PANGEBIN] vs [PBF on Skesa]: Precision - Recall - F1 scores for {samples} samples")

mixed.save("scatter_pangebin_skesa.pdf", ppi=300)
mixed


NameError: name 'skesa_stats' is not defined

In [12]:
charts["prec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_uni:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='precision')
)


charts["rec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_uni:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='recall')
)

charts["f1"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='pbf_uni:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='f1')
)

mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"[PANGEBIN] vs [PBF on Unicycler]: Precision - Recall - F1 scores for {samples} samples")
mixed.save("scatter_pangebin_unicycler.pdf", ppi=300)
mixed


alt.HConcatChart(...)

In [13]:
charts["prec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='gplas:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='precision')
)


charts["rec"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='gplas:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='recall')
)

charts["f1"] = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='gplas:Q',
    y='pangebin:Q',
    color='metric:N'
).transform_filter(
     alt.FieldEqualPredicate(field='metric', equal='f1')
)

pangeplas_all = alt.Chart(df_scatter).mark_circle(size=60).encode(
    x='gplas:Q',
    y='pangebin:Q',
    color='metric:N'
)

mixed = alt.hconcat(charts["prec"], charts["rec"], charts["f1"]).properties(title=f"[PANGEBIN] vs [GPLAS on Unicycler]: Precision - Recall - F1 scores for {samples} samples")
mixed.save("scatter_pangebin_gplas.pdf", ppi=300)
mixed


alt.HConcatChart(...)

In [14]:
pangeplas_all.save("scatter_pangeplas_all.pdf", ppi=300)
pangeplas_all



alt.Chart(...)

In [73]:
# SECOND SCATTER SERIES

header = ["sample", "precision", "recall", "tool"]
df_scatter2 = pd.DataFrame(columns=header)
for i in range(len(skesa_stats["prec"])):
    df_scatter2.loc[len(df_scatter2)] = {"sample":i,"precision":uni_stats["prec"][i], "recall":uni_stats["rec"][i],                                       "tool":"unicycler"}
    df_scatter2.loc[len(df_scatter2)] = {"sample":i,"precision":skesa_stats["prec"][i], "recall":skesa_stats["rec"][i], "tool":"skesa"}
    df_scatter2.loc[len(df_scatter2)] = {"sample":i,"precision":pange_mod1["prec"][i], "recall":pange_mod1["rec"][i], "tool":"pangenome"}
    df_scatter2.loc[len(df_scatter2)] = {"sample":i, "precision": gplas_stats["prec"][i], "recall":gplas_stats["rec"][i], "tool": "gplas"}
    

In [74]:
pangeplas = alt.Chart(df_scatter2).mark_circle(size=60).encode(
    y='precision:Q',
    x='recall:Q',
    color='tool:N'
).transform_filter(
     alt.FieldOneOfPredicate(field='tool', oneOf=['pangenome','gplas'])
)
pangeplas = pangeplas.properties(title=f"Precision/Recall scatterplot for [PANGEBIN] and [GPLAS] on {samples} samples")

pangeplas.save("scatter_pangeplas_vs.pdf", ppi=300)
pangeplas

alt.Chart(...)

In [75]:
charts["prec"] = alt.Chart(df_scatter2).mark_circle(size=60).encode(
    y='precision:Q',
    x='recall:Q',
    color='tool:N'
).transform_filter(
     alt.FieldEqualPredicate(field='tool', equal='pangenome')
)


charts["rec"] = alt.Chart(df_scatter2).mark_circle(size=60).encode(
    y='precision:Q',
    x='recall:Q',
    color='tool:N'
).transform_filter(
     alt.FieldEqualPredicate(field='tool', equal='skesa')
)

charts["f1"] = alt.Chart(df_scatter2).mark_circle(size=60).encode(
    y='precision:Q',
    x='recall:Q',
    color='tool:N'
).transform_filter(
     alt.FieldEqualPredicate(field='tool', equal='unicycler')
)
all = alt.Chart(df_scatter2).mark_circle(size=60).encode(
    y='precision:Q',
    x='recall:Q',
    color='tool:N'
)


mixed = alt.vconcat(alt.hconcat(charts["prec"], all), alt.hconcat( charts["rec"], charts["f1"])).properties(title=f"Precision/Recall scatterplot for [PANGEBIN], [PBF on Skesa] and [PBF on Unicycler] on {samples} samples")
mixed.save("scatter_metrics.pdf", ppi=300)
mixed

alt.VConcatChart(...)

In [35]:
print(gplas_stats['prec'].sort())



None


In [36]:
gplas_stats

{'sample': ['kpne-SAMEA3357080',
  'kpne-SAMN07312490',
  'kpne-SAMEA3357223',
  'kpne-SAMEA3357043',
  'kpne-SAMEA3357010',
  'ecol-SAMN04014852',
  'kpne-SAMN03024573',
  'ecol-SAMN04014855',
  'kpne-SAMN02786856',
  'kpne-SAMN16357457',
  'kpne-SAMN16357468',
  'kpne-SAMN16357450',
  'kpne-SAMN11056346',
  'kpne-SAMN16357466',
  'kpne-SAMN16357461',
  'ecol-SAMN04014847',
  'erog-SAMN10163217',
  'kpne-SAMEA3357328',
  'kpne-SAMN16357467',
  'kpne-SAMN16357458',
  'kpne-SAMN16357451',
  'kpne-SAMN16357456',
  'kpne-SAMN11056490',
  'kpne-SAMN07312483',
  'ehor-SAMN02713646',
  'ehor-SAMN02713683',
  'kpne-SAMEA3357077',
  'kpne-SAMEA3357272',
  'kpne-SAMN16357470',
  'kpne-SAMN16357448',
  'ecol-SAMN04014858',
  'ecol-SAMN04014856',
  'kpne-SAMN03024583',
  'kpne-SAMN16357449',
  'ecol-SAMN02991226',
  'kpne-SAMN16357471',
  'ebug-SAMN16357569',
  'kpne-SAMN16357245',
  'kpne-SAMN16357453',
  'kpne-SAMN11056345',
  'kpne-SAMN16357465',
  'eclo-SAMN16357573',
  'kpne-SAMN16357464',
 

In [37]:
pange_mod1['prec'].sort()

In [38]:
pange_mod1['prec']

[0.542913368638528,
 0.5716721244205368,
 0.5875896580878502,
 0.6436082026448816,
 0.649001539870867,
 0.6512448553282573,
 0.6576276687764242,
 0.662337826033798,
 0.6692689983196921,
 0.697044101654712,
 0.7102961050082288,
 0.7413014179453123,
 0.742894661760295,
 0.7477353835790691,
 0.7610224156036764,
 0.7725333218554068,
 0.7895175360289672,
 0.8054395588897187,
 0.8104139957717498,
 0.8146572267709902,
 0.8362401362225329,
 0.8777603440260344,
 0.8999046554445029,
 0.913761284327888,
 0.9147924581471153,
 0.9224685171754466,
 0.9273445584118946,
 0.9468923592401368,
 0.9582104864384087,
 0.9865314954653555,
 0.9963232519481927,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [43]:
print(df)

                sample       tool  precision    recall        f1
0    kpne-SAMEA3357080      gplas   0.977044  0.760248  0.855120
1    kpne-SAMEA3357080  unicycler   0.230568  0.969423  0.372533
2    kpne-SAMEA3357080      skesa   0.664926  0.522048  0.584888
3    kpne-SAMEA3357080  pangenome   0.761022  0.392424  0.517828
4    kpne-SAMN07312490      gplas   0.982785  0.821278  0.894802
5    kpne-SAMN07312490  unicycler   0.524305  0.979399  0.682986
6    kpne-SAMN07312490      skesa   0.433955  0.766756  0.554235
7    kpne-SAMN07312490  pangenome   0.657628  0.630484  0.643770
8    kpne-SAMEA3357223      gplas   0.248411  0.969609  0.395497
9    kpne-SAMEA3357223  unicycler   0.096440  0.914216  0.174474
10   kpne-SAMEA3357223      skesa   0.180643  0.917906  0.301877
11   kpne-SAMEA3357223  pangenome   1.000000  0.828770  0.906369
12   kpne-SAMEA3357043      gplas   1.000000  0.829289  0.906679
13   kpne-SAMEA3357043  unicycler   0.732712  1.000000  0.845740
14   kpne-SAMEA3357043   

In [40]:
pd.set_option('display.max_rows', None)


In [41]:
dfbcp = df

In [42]:
df.loc[(df['f1'] == 0.0) & (df['tool'] == 'pangenome')]

,sample,tool,precision,recall,f1


In [44]:
df.loc[(df['precision'] == 0.0) & (df['tool'] == 'pangenome')]


,sample,tool,precision,recall,f1


In [45]:
df.loc[(df['recall'] == 0.0) & (df['tool'] == 'pangenome')]


,sample,tool,precision,recall,f1


In [46]:
df.loc[(df['f1'] == 0.0) & (df['tool'] == 'skesa')]


,sample,tool,precision,recall,f1
182,ehor-SAMN16357577,skesa,0.0,0.0,0.0


In [47]:
df.loc[(df['f1'] == 0.0) & (df['tool'] == 'unicycler')]


,sample,tool,precision,recall,f1
181,ehor-SAMN16357577,unicycler,0.0,0.0,0.0


In [48]:
df.loc[(df['recall'] == 0.0) & (df['tool'] == 'unicycler')]


,sample,tool,precision,recall,f1
181,ehor-SAMN16357577,unicycler,0.0,0.0,0.0


In [49]:
df.loc[(df['sample'].str.contains('abau')) & (df['tool'] == 'unicycler')]

,sample,tool,precision,recall,f1


In [50]:
# Print samples where gplas goes better

df.loc[(df['tool'].isin(["gplas", "pangenome"])) & (df['f1']< 0.5) ]
df.loc[(df['recall']) < 0.5 ].head(5)

,sample,tool,precision,recall,f1
3,kpne-SAMEA3357080,pangenome,0.761022,0.392424,0.517828
22,ecol-SAMN04014852,skesa,0.935520,0.178262,0.299462
23,ecol-SAMN04014852,pangenome,0.914792,0.419449,0.575172
30,ecol-SAMN04014855,skesa,0.898961,0.163007,0.275972
31,ecol-SAMN04014855,pangenome,1.000000,0.242213,0.389970


In [35]:
df.loc[
( (df["recall"] > 0.7 )& (df['tool'] == 'gplas') ) | ((df['tool'] == 'pangenome') & (df['recall'] < 0.7))
]


,sample,tool,precision,recall,f1
0,kpne-SAMEA3357080,gplas,0.977044,0.760248,0.855120
3,kpne-SAMEA3357080,pangenome,0.761022,0.392424,0.517828
4,kpne-SAMN07312490,gplas,0.982785,0.821278,0.894802
7,kpne-SAMN07312490,pangenome,0.657628,0.630484,0.643770
8,kpne-SAMEA3357223,gplas,0.248411,0.969609,0.395497
12,kpne-SAMEA3357043,gplas,1.000000,0.829289,0.906679
16,kpne-SAMEA3357010,gplas,1.000000,0.829670,0.906907
20,ecol-SAMN04014852,gplas,0.871354,0.808014,0.838489
23,ecol-SAMN04014852,pangenome,0.914792,0.419449,0.575172
24,kpne-SAMN03024573,gplas,0.992047,0.735495,0.844721


In [36]:
## Check numerosity of ambigous contigs inside each pangenome graph ...
## 

In [52]:
pan_csv = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome.transformed.mix.csv"
uni_csv = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/unicycler.transformed.mix.csv"
ske_csv = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/skesa.transformed.mix.csv"

In [53]:
pan_df = pd.read_csv(pan_csv, sep = ",")
pan_df.head(4)
uni_df = pd.read_csv(uni_csv, sep = ",")
ske_df = pd.read_csv(ske_csv, sep = ",")


In [54]:
df_s = pan_df
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] != "plasmid")].count()


contig         562
label          562
pred_label       0
length         562
type           562
plasmids       562
chromosomes     38
bins             0
dtype: int64

In [55]:
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] == "plasmid")].count()

contig         166
label          166
pred_label     166
length         166
type           166
plasmids       166
chromosomes     27
bins           166
dtype: int64

In [56]:
df_s = uni_df
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] != "plasmid")].count()


contig         12
label          12
pred_label      0
length         12
type           12
plasmids       12
chromosomes     0
bins            0
dtype: int64

In [57]:
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] == "plasmid")].count()

contig         114
label          114
pred_label     114
length         114
type           114
plasmids       114
chromosomes     34
bins           114
dtype: int64

In [58]:
df_s = ske_df
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] != "plasmid")].count()


contig         46
label          46
pred_label      0
length         46
type           46
plasmids       46
chromosomes     7
bins            0
dtype: int64

In [59]:
df_s.loc[ (df_s["label"].isin(["plasmid", "ambiguous"])) & 
           (df_s["pred_label"] == "plasmid")].count()

contig         63
label          63
pred_label     63
length         63
type           63
plasmids       63
chromosomes     8
bins           63
dtype: int64

In [60]:
df.loc[df['sample'] == "ecol-SAMN02991226"]

,sample,tool,precision,recall,f1
136,ecol-SAMN02991226,gplas,0.611322,0.359165,0.452485
137,ecol-SAMN02991226,unicycler,0.352065,0.483031,0.407278
138,ecol-SAMN02991226,skesa,0.687608,0.149329,0.245371
139,ecol-SAMN02991226,pangenome,0.946892,0.190609,0.317338


In [61]:
### Now check IF the ground truth in SKESA and UNICYCLER is matching or not..

## READ (gfa) (unicycler gt) (skesa gt)  and check for ambigous labeling 
gt_csv = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/ecol-SAMN02991226.gt.csv"
gt = pd.read_csv(gt_csv, sep=",")


In [62]:
gt.head()

,contig,label,length,plasmids,chromosomes
0,ske_1,chromosome,92437.0,NaN,chr_1
1,ske_10,chromosome,601.0,NaN,chr_1
2,ske_100,chromosome,1835.0,NaN,chr_1
3,ske_101,chromosome,18594.0,NaN,chr_1
4,ske_102,chromosome,97438.0,NaN,chr_1


In [63]:
gt.groupby(["label"]).sum("length")


,length
label,
ambiguous,29943.0
chromosome,10078251.0
plasmid,542860.0
short,2243.0
undef,40882.0


In [64]:
gt.groupby("label").count()


,contig,length,plasmids,chromosomes
label,,,,
ambiguous,49,49,49,49
chromosome,588,588,0,588
plasmid,186,186,186,0
short,176,176,0,0
undef,11,11,0,0


In [65]:
gt_bcp = gt
import gfapy as gp
import numpy as np
gfa = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/ecol-SAMN02991226.enanched.pangenome.gfa"
gfa = gp.Gfa.from_file(gfa)
gfa.validate()

In [122]:
gt = gt_bcp
header = ["contig","label","length","plasmids","chromosomes", "type"]
gt_transformed = pd.DataFrame(columns=header)
for seg in gfa.segments:
    contig_list = seg.cl.split(",")## retrieve the GT of each contig
    label = None
    ch_list = []
    pl_list = []
    for contig in contig_list:
        new_label = None
        if contig in gt['contig'].values: # if the contig is in the ground truth file
            assert(gt.loc[gt["contig"] == contig, 'label'].count() == 1), f"more than one label for contig???{contig}"
            new_label = gt.loc[gt['contig'] == contig, 'label'].values[0]# retrieve the label
            assert(gt.loc[gt["contig"] == contig, 'chromosomes'].count() <= 1), f"more than one CHROMOSOME for contig???{contig}"
            chromos = gt.loc[gt['contig'] == contig, 'chromosomes'].values[0]

            if chromos is not np.nan:
                for x in (gt.loc[gt['contig'] == contig, 'chromosomes'].values[0]).split(","):
                    ch_list.append(x)
                    
            assert(gt.loc[gt["contig"] == contig, 'plasmids'].count() <= 1), f"more than one CHROMOSOME for contig???{contig}"
            plas = gt.loc[gt['contig'] == contig, 'plasmids'].values[0]
            
            if plas is not np.nan:
                for x in (gt.loc[gt['contig'] == contig, 'plasmids'].values[0]).split(","):
                    pl_list.append(x)
                    
                    
            if new_label in ["plasmid", "chromosome", "ambiguous"]: 
                if label is None or label == "undef": # if this is the first label, assign it
                    label = new_label
                elif label != new_label: # if it is not the first label, is ambiguous if it differs from the previous one
                    label = "ambiguous"
                elif label == new_label: # (this assignment is not necessary, but it is more explicit)
                    label = new_label
            else:
                continue # do not annotate [short] and [undef] label
        else:
            assert(False), f"Contig {contig} not found in the ground truth file"
        
        if new_label is None:
            label = "undef"
    ch_list = list(set(ch_list))
    pl_list = list(set(pl_list))
    ch_list = ",".join(ch_list)
    pl_list = ",".join(pl_list)
    """Add the new segment(fragment) information to the transformed ground truth"""
    gt_transformed.loc[len(gt_transformed)] = {'contig': seg.name, 'label': label, 'length': seg.LN, 'plasmids':pl_list, 'chromosomes': ch_list,  'type': seg.aa }
    if (True):
        tlist = []
        for contig in contig_list:
            tlist.append(gt.loc[gt["contig"] == contig]['label'].values[0])
        if (len(set(tlist))) != 1:
            print(contig_list)
            tlist = set(tlist)
            print(tlist)
            if "ambiguous" in tlist:
                tlist.remove("ambiguous")
                new_lab = str(next(iter(tlist)))
                for ct in contig_list:
                    gt["label"] = gt["label"].where(gt["contig"].ne(ct), new_lab)
                print(contig_list)
                print(tlist)
                print("------")
            else:
                print("UNASSIGNIGN DISCREPANCY: ")
                print(gt.loc[gt["contig"].isin(contig_list)])
                print(seg.name, seg.LN)
                print(contig_list)
                print(tlist)
                print("assigning longer label to contig")
                selected_id = gt.loc[(gt["contig"].isin(contig_list))]['length'].idxmax()
                new_lab = gt.at[selected_id, "label"]
                for ct in contig_list:
                    gt["label"] = gt["label"].where(gt["contig"].ne(ct), new_lab)
                print(contig_list)
                print(tlist)
                print("------")
                
                
        

In [123]:
gt.head()

,contig,label,length,plasmids,chromosomes
0,ske_1,chromosome,92437.0,NaN,chr_1
1,ske_10,chromosome,601.0,NaN,chr_1
2,ske_100,chromosome,1835.0,NaN,chr_1
3,ske_101,chromosome,18594.0,NaN,chr_1
4,ske_102,chromosome,97438.0,NaN,chr_1


In [ ]:
## Strategy --- if ambiguous  skesa and chromo unicycler, Merge them

## IF PLASMID vs CHROMOSOME, check length and maybe make UNDEF or OPPOSITE

In [56]:
gt_transformed.loc[gt_transformed['label']=="plasmid"].groupby(['type']).sum('length')

,length
type,
b,232157
s,43318
u,23221


In [57]:
gt_transformed.loc[gt_transformed['label']=="chromosome"].groupby(['type']).sum('length')

,length
type,
b,4978072
s,63677
u,49455


In [58]:
gt_transformed.loc[gt_transformed['label']=="ambiguous"].groupby(['type']).sum('length')

,length
type,
b,7257
s,6524
u,8738


In [125]:
gt_transformed.head()

,contig,label,length,plasmids,chromosomes,type
0,1,chromosome,600,,chr_1,s
1,2,chromosome,713,,chr_1,b
2,3,chromosome,1,,chr_1,s
3,4,plasmid,847,pl_3,,b
4,5,chromosome,807,,chr_1,s


In [128]:
# Read in the bin file
bin_fp = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/result/ecol-SAMN02991226.pangenome.bins.tsv"
"""#Pls_ID	Flow	GC_bin	Length	Contigs"""
bin_file = pd.read_csv(bin_fp, sep='\t')

header = ["contig", "label", "length", "plasmids", "type"]
bin_transformed = pd.DataFrame(columns=header)
## if bin_file has no rows
for _, row in bin_file.iterrows():
        contigs = [x.split(":")[0] for x in row['Contigs'].split(',')]
        for contig in contigs:
            type_ = gfa.segment(contig).aa # if str(args.type).startswith("pangenome") else str(args.type)[0]
            bin_transformed.loc[len(bin_transformed)] = {'contig': contig, 'label': "plasmid", 'plasmids': row['#Pls_ID'], 'length': gt_transformed.loc[gt_transformed['contig'] == contig, 'length'].values[0], 'type': type_}

In [132]:
bin_transformed.sort_values(by='contig', inplace=True)
bin_transformed = bin_transformed.groupby('contig').agg('sum').reset_index()
bin_transformed.drop_duplicates(subset='contig', keep='first', inplace=True)

for index, row in bin_transformed.iterrows():
    bin_transformed.loc[index, 'label'] = "plasmid" if row['label'].startswith('plasmid') else row['label']

pred_plasmids_len = bin_transformed.groupby('label').sum()['length']['plasmid']


In [133]:
mix_transformed_header = ["contig", "label", "pred_label", "length", "type", "plasmids", "chromosomes", "bins"]
    
mix_transformed = pd.DataFrame(columns=mix_transformed_header)
mix_transformed = pd.concat([mix_transformed, gt_transformed], ignore_index=True)

In [136]:
argsout = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome2.transformed.mix.csv"
for _, row in bin_transformed.iterrows():
    if row['contig'] in mix_transformed['contig'].values:
        mix_transformed.loc[mix_transformed['contig'] == row['contig'], 'pred_label'] = row["label"]
        mix_transformed.loc[mix_transformed['contig'] == row['contig'], 'bins'] = row["plasmids"]     
mix_transformed.to_csv(f"{argsout}", sep=',', index=False, encoding='utf-8')

In [137]:
mix_csv = argsout
evalgt = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome2.eval.gt.csv"
evalbin = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome2.eval.bin.csv"
evalbinMOD= "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome2.eval.bin.mod1.csv"



evalout = "/Users/msgro/code_new/output_robertson/ecol-SAMN02991226/output/pangenome2.eval.out"

!python pangebin/bin/convert_pred.py --input {mix_csv} --output-gt {evalgt} --output-bin {evalbin}

!python3 pangebin/bin/modify_bins.py --mod 1 --bins {evalbin} --gtruth {evalgt} --output {evalbinMOD}

!python PlasEval/src/plaseval.py eval --pred {evalbinMOD} --gt {evalgt} --out_file {evalout} --min_len 1

row_bins: P14
bins: ['P14']
row_bins: P3P19P10
bins: ['P3', 'P19', 'P10']
row_bins: P10P3P19
bins: ['P10', 'P3', 'P19']
row_bins: P19P3P10
bins: ['P19', 'P3', 'P10']
row_bins: P65
bins: ['P65']
row_bins: P36
bins: ['P36']
row_bins: P63
bins: ['P63']
row_bins: P63
bins: ['P63']
row_bins: P8
bins: ['P8']
row_bins: P60
bins: ['P60']
row_bins: P45
bins: ['P45']
row_bins: P7P3
bins: ['P7', 'P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P3
bins: ['P3']
row_bins: P55
bins: ['P55']
row_bins: P55
bins: ['P55']
row_bins: P27
bins: ['P27']
row_bins: P27
bins: ['P27']
row_bins: P3P10P19
bins: ['P3', 'P10', 'P19']
row_bins: P10P3
bins: ['P10', 'P3']
row_bins: P25P3P10
bins: ['P25', 'P3', 'P10']
row_bins: P3
bins: ['P3']
row_bins: P3P4
bins: ['P3', 'P4']
row_bins: P64
bins: ['P64']
row_bins: P50
bins: ['P50']
row_bins: P21
bins: ['P21']
row_bins: P9
bins: ['P9']
row_bins: P9
b